To use the examples in this chapter, first run the code below to import the PyCh Library.

In [1]:
from PyCh import *

PyCh version 0.3 imported succesfully.
 


# 9 Buffers
We ended the previous chapter with a model that illustrates how the buffer functions. In this chapter we will discuss the various types of buffers.

Using channels, entities can be transfered from one process to the next, in a synchronous manner (Sender and receiver perform the communication at exactly the same moment in time, and the communication is instantaneous). In many systems however, processes do not use synchronous communication, they use asynchronous communication instead. Entities (products, packets, messages, simple tokens, and so on) are sent, temporarily stored in a buffer, and then received.

In fact, the decoupling of sending and receiving is very important, it allows compensating temporarily differences between the number of items that are sent and received (Under the assumption that the receiver is fast enough to keep up with the sender in general, otherwise the buffer will grow forever or overflow).

For example, consider the exchange of items from a producer process `P` to a consumer process `C` as shown in the figure below. In the unbuffered situation, both processes communicate at the same time.
This means that when one process is (temporarily) faster than the other, it has to wait for the other process before communication can take place.

| Figure 9.1: A producer and a consumer |
-
<img src="figures/8-1.png" width=75%>
<a id='fig:9-1'></a>

As shown in the example below, when the consumer needs more time to process entities than the producer, then the producer will have to wait for the consumer.

In [2]:
@process
def producer(env, c_out):
    for i in range(5):
        t_ready = env.now
        yield env.execute(c_out.send(i))
        t_waiting = env.now - t_ready
        print(f"The producer has sent entity {i} at t = {env.now}, and had to wait {t_waiting} seconds.")
        yield env.timeout(1)

@process
def consumer(env, c_in):
    while True:
        i = yield env.execute(c_in.receive())
        yield env.timeout(3)

def model():
    env = Environment()
    a = Channel(env)
    P = producer(env, a)
    C = consumer(env, a)
    env.run()

model()

The producer has sent entity 0 at t = 0, and had to wait 0 seconds.
The producer has sent entity 1 at t = 3.0, and had to wait 2.0 seconds.
The producer has sent entity 2 at t = 6.0, and had to wait 2.0 seconds.
The producer has sent entity 3 at t = 9.0, and had to wait 2.0 seconds.
The producer has sent entity 4 at t = 12.0, and had to wait 2.0 seconds.


With a buffer in-between, the producer can give its item to the buffer, and continue with its work. Likewise, the consumer can pick up a new item from the buffer at any later time (if the buffer has items). In this simulation tool, buffers are not modeled as channels, they are modeled as additional processes instead. The result is shown in Figure 9.2 below.

| Figure 9.2: A producer and a consumer, with an additional buffer process. |
-
<img src="figures/9-2.png" width=75%>
<a id='fig:9-2'></a>

The producer sends its items synchronously (using channel `a`) to the buffer process. The buffer process keeps the item until it is needed. The consumer gets an item synchronously (using channel `b`) from the buffer when it needs a new item (and one is available).

In manufacturing networks, buffers, in combination with servers, play a prominent role, for buffering items in the network. Various buffer types exist in these networks: buffers can have a finite or infinite capacity, they have a input/output discipline, for example a first-out queuing discipline or a priority-based discipline. Buffers can store different kinds of items, for example, product-items, information-items, or a combination of both. Buffers may also have sorting facilities, etc.

In this chapter some buffer types are described, and with the presented concepts numerous types of buffer can be designed by the engineer. First a simple buffer process with one buffer position is presented, followed by more advanced buffer models.
The producer and consumer processes are not discussed in this chapter.

## 9.1 A one-place buffer
A buffer usually has a receiving channel and a sending channel, for receiving and sending items. A buffer, buffer `B1`, is presented in Figure 9.3.

| Figure 9.3: A 1-place buffer. |
-
<img src="figures/9-3.png" width=75%>
<a id='fig:9-3'></a>

The simplest buffer is a one-place buffer, for buffering precisely one item. A one-place buffer is shown below.  `c_in` and `c_out` are the receiving and sending channels. Entity `i` is buffered in the process. A buffer receives an item, stores the item, and sends the item to the next process, if the next process is willing to receive the item. The buffer is not willing to receive a second item, as long as the first item is still in the buffer. 

In [3]:
@process
def buffer(env, c_in, c_out):
    while True:
        i = yield env.execute(c_in.receive())
        yield env.execute(c_out.send(i))

def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    P = producer(env, a)
    B1 = buffer(env, a, b)
    C = consumer(env, b)
    env.run()

model()

The producer has sent entity 0 at t = 0, and had to wait 0 seconds.
The producer has sent entity 1 at t = 1, and had to wait 0 seconds.
The producer has sent entity 2 at t = 3.0, and had to wait 1.0 seconds.
The producer has sent entity 3 at t = 6.0, and had to wait 2.0 seconds.
The producer has sent entity 4 at t = 9.0, and had to wait 2.0 seconds.


---
A two-place buffer can be created, by using the one-place buffer process twice. A two-place buffer is depicted below.

| Figure 9.4: A 2-place buffer. |
-
<img src="figures/9-4.png" width=75%>
<a id='fig:9-4'></a>

A two-place buffer is shown below. In the two-place buffer, processes `B1` and `B2` buffer maximal two items.
If each buffer process contains an item, a third item has to wait in front of process `B1`. 

Note that `buffer_2_place` is not a process but a submodel, because it does not yield any events by itself

In [4]:
def buffer_2_place(env, c_in, c_out):
    c_B1B2 = Channel(env)
    B1 = buffer(env, c_in, c_B1B2)
    B2 = buffer(env, c_B1B2, c_out)
    
def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    P = producer(env, a)
    B = buffer_2_place(env, a, b)
    C = consumer(env, b)
    env.run()

model()

The producer has sent entity 0 at t = 0, and had to wait 0 seconds.
The producer has sent entity 1 at t = 1, and had to wait 0 seconds.
The producer has sent entity 2 at t = 2, and had to wait 0 seconds.
The producer has sent entity 3 at t = 3.0, and had to wait 0.0 seconds.
The producer has sent entity 4 at t = 6.0, and had to wait 2.0 seconds.


---
This procedure can be extended to create even larger buffers. Another, more preferable manner however, is to describe a buffer in a single process by using a `select` statement and a list for storage of the items. Such a buffer is discussed in the next section.

## 9.2 A single process buffer
An informal description of the process of a buffer, with an arbitrary number of stored items, is the following:
- If the buffer has space for an item, *and* can receive an item from another process via channel `a`,
    the buffer process receives that item, and stores the item in the buffer.
- If the buffer contains at least one item, *and* the buffer can send that item to another process via
    channel `b`, the buffer process sends that item, and removes that item from the buffer.
- If the buffer can both send and receive a value, the buffer process selects one of the two possibilities (in a non-deterministic manner).
- If the buffer can not receive an item, and can not send an item, the buffer process waits.

Next to the sending and receiving of items (to and from the buffer process) is the question of how to order the stored items.
A common form is the *first-in first-out* (fifo) queuing discipline.
Items that enter the buffer first (first-in) also leave first (first-out), the order of items is preserved by the buffer process.

In the model of the buffer, list `xs` is used for storing the received items.
New item `x` is added at the rear of list by the statement:

    xs = xs+ [x]
    
When the first item is sent, it is removed from the list with:
    
    xs = xs[1:]
    

In [51]:
@process
def Generator(env, c_out):
    for x in range(5):
        yield env.execute(c_out.send(x))

@process
def Server(env, c_in, c_out, t_processing):
    while True:
        x = yield env.execute(c_in.receive())
        yield env.timeout(t_processing)
        yield env.execute(c_out.send(x))

@process
def Exit(env, c_in):
    while True:
        x = yield env.execute(c_in.receive())

In [52]:
@process
def Buffer(env, a, b):
    xs = [];
    while True:
        print("The buffer contains", len(xs), "item(s)")
        receiving = a.receive()
        guard_sending = len(xs)>0                             # The guard function for the sending event
        sending = b.send(xs[0]) if guard_sending else None    # If the guard is not satisfied, sending is defines as None
        x = yield env.select(receiving, sending)
        if selected(receiving):
            xs = xs + [x]
        if selected(sending):
            xs = xs[1:]



@process
def Exit(env, b):
    while True:
        x = yield env.execute(b.receive())
               
def model():
    env = Environment()
    a = Channel(env)
    b = Channel(env)
    c = Channel(env)
    G = Generator(env, a)
    B = Buffer(env, a, b)
    S = Server(env, b, c)
    E = Exit(env, c)
    env.run()
    
model()

TypeError: Server() missing 1 required positional argument: 't_processing'